In [2]:
import numpy as np
from matplotlib import pyplot as plt

In [ ]:
# the "FAD frame" is:
#  +x from C10 to C9A
#  +y from N5 to N10
#  +z completing the right hand rule

TDM_IN_FAD_FRAME = np.array([])

In [ ]:
def rotation_matrix_crystal(trj, chain=0):
    return

def rotation_matrix_from_A(A):
    return

def expand_p212121(vector):
    return set_of_vectors

### TDM

Logic. Given TDM $\mu$ in the FAD frame of reference, we need to compute the set of TDMs in the experimental frame. To do this, we:

1. for each chain: compute the rotation matrices to go from the crystal frame to the frame of FAD (2 rotation matrices)
2. compute the TDM for each chain in the crystal frame (2 vectors)
3. symmetry-expand these vectors (4 + 4 vectors)
4. rotate these vectors using the A-matrix into the experimental frame

Then we can simply compute the component of the TDM that is in the (circular) polarization plane as:

$$
\sum_i 1 - (\vec{\mu}_i \cdot \vec{z})^2
$$

In [7]:
a_matrix_file = '/Users/tjlane/Desktop/PL-workshop/signals_chain_A_vs_B/orient/crystal_Amatrices_100us_dark.npy'
#a_matrix_file = '/Users/tjlane/Desktop/PL-workshop/signals_chain_A_vs_B/orient/crystal_Amatrices_TD_all.npy'
#a_matrix_file = '/Users/tjlane/Desktop/PL-workshop/signals_chain_A_vs_B/orient/crystal_Amatrices_100us_l+d.npy'
 
A_matrices = np.load(a_matrix_file)
print(A_matrices.shape)

(115492, 3, 3)


In [9]:
A_matrices[0]

array([[ -0.39042838,  -4.21037955,  -5.55653905],
       [  2.80528735,  -9.22269548,   6.81339931],
       [-16.53159563,  -2.69372133,   3.02869369]])

In [14]:
# this is a test to ensure we slice correctly
# the code below is the same as used to compute A_matrices

a_star = np.array([1, 1, 0])
b_star = np.array([0, 2, 0])
c_star = np.array([0, 0, 3])

A_star = np.column_stack((a_star, b_star, c_star))
print(A_star)

A = np.linalg.inv(A_star)
print(A[:,0]) # this is "x"

[[1 0 0]
 [1 2 0]
 [0 0 3]]
[ 1.  -0.5  0. ]
